In [66]:
class OptionPricing:
    def __init__(self, stock_name, strike_price, ttm, option_type = 'Call'): #visualizaition = True):
        self.stock_name = stock_name
        self.option_type = option_type
        #self.visual = visualization
        self.strike_price = strike_price
        self.ttm = ttm
        
    
    def get_stock_info(self, start = 0, end = 0):

        import pandas as pd
        import numpy as np
        import datetime
        import pandas_datareader.data as web
        import fix_yahoo_finance as yf
        end = datetime.date.today()
        start = end - datetime.timedelta(days = 252*10)
        try:
            data = web.get_data_yahoo(self.stock_name, start,end)
            data['Return'] = data['Close'].pct_change()
            data = data.dropna()
            self.std = data['Return'].describe().loc['std']
            self.std = self.std * np.sqrt(252) #annualize
            self.price = data['Close'].iloc[len(data)-1]

            return self.price, self.std

        except:
            print('Cannot find stock data')
    
    
    
    def get_rf(self, date = 0, tenor = '10 yr'):
        import pandas as pd
        import requests
        from bs4 import BeautifulSoup
        import datetime
        url = 'https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yield'
        response = requests.get(url)

        if response.status_code == 200:
            data_page = BeautifulSoup(response.content,'lxml')
            yield_table = data_page.find('table', class_ = 't-chart')
            trs = yield_table.find_all('tr')
            col = [trs[0].find_all('th', scope = 'col')[i].get_text() 
                    for i in range(1,len(trs[0].find_all('th', scope = 'col')))]
            index = [trs[i].find_all('td', scope = 'row')[0].get_text() for i in range(1,len(trs))]

            rates = [[float(trs[i].find_all('td')[j].get_text()) for j in range(1,len(col)+1)] for i in range(1,len(trs))]
            data = pd.DataFrame(rates, columns = col, index = index)
            data = data/100 #pct_value

            date = data.index.values[len(data)-1] #latest data

            self.rf_rate = data.loc[date][tenor]
        else:
            print('Failure')

        return self.rf_rate #risk free rate

    
    def BS_model(self):
        import scipy.stats
        import numpy as np
        from math import sqrt, exp
        import pandas as pd

        stock_price, var = self.get_stock_info() # from get_stock_info
        strike_price = self.strike_price
        r_f = self.get_rf()
        ttm =self.ttm
        
        norm_dist = scipy.stats.norm(0, 1)
        d1 = 1/(var * sqrt(ttm)) * (np.log(stock_price/strike_price) +(r_f + (var**2/2))*ttm)
        d2 = d1 - var*sqrt(ttm)

        call_value = norm_dist.cdf(d1) * stock_price - norm_dist.cdf(d2) * strike_price * exp(-r_f * ttm)
        put_value = strike_price * exp(-r_f * ttm) - stock_price + call_value

        delta_call = norm_dist.cdf(d1)
        delta_put = norm_dist.cdf(d1) - 1
        gamma = norm_dist.pdf(d1)/(stock_price * var * sqrt(ttm))
        vega = stock_price * norm_dist.pdf(d1) * sqrt(ttm)
        theta_call = -(stock_price * norm_dist.pdf(d1) * var)/(2*sqrt(ttm))- (r_f * strike_price * 
                                                                              norm_dist.cdf(d2) * exp(-r_f*ttm))
        theta_put = -(stock_price * norm_dist.pdf(d1) * var)/(2*sqrt(ttm))+ (r_f * strike_price * 
                                                                              norm_dist.cdf(-d2) * exp(-r_f*ttm))
        rho_call = strike_price * ttm * norm_dist.cdf(d2) * exp(-r_f * ttm)
        rho_put = -strike_price * ttm * norm_dist.cdf(-d2) * exp(-r_f * ttm)

        price = [call_value, put_value]
        delta = [delta_call, delta_put]
        gamma = [gamma]*2
        vega = [vega]*2
        theta = [theta_call, theta_put]
        rho = [rho_call, rho_put]

        output = [price, delta, gamma, vega, theta, rho]
        col = ['Call', 'Put']
        index = ['Price', 'Delta', 'Gamma', 'Vega', 'Theta', 'Rho']

        self.BS_df = pd.DataFrame(output, columns = col, index = index)
        
        return self.BS_df 

        
    
    def BS_sensitivity_plot(self):
#         stock_price = self.price
#         var = self.std ** 2
#         strike_price = self.strike_price
#         r_f = self.r_f
#         ttm =self.ttm
        option_type = self.option_type
        
        
        
    def mc_euro_options(self, option_type,S,K,T,r,v,simulations):
        
        
            data['Return'] = data['Close'].pct_change()
            data = data.dropna()
            self.std = data['Return'].describe().loc['std']
            self.std = self.std * np.sqrt(252) #annualize
            self.price = data['Close'].iloc[len(data)-1]

            return self.price, self.std

        except:
            print('Cannot find stock data')
    
    
    
    def get_rf(self, date = 0, tenor = '10 yr'):
        import pandas as pd
        import requests
        from bs4 import BeautifulSoup
        import datetime
        url = 'https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yield'
        response = requests.get(url)

        if response.status_code == 200:
            data_page = BeautifulSoup(response.content,'lxml')
            yield_table = data_page.find('table', class_ = 't-chart')
            trs = yield_table.find_all('tr')
            col = [trs[0].find_all('th', scope = 'col')[i].get_text() 
                    for i in range(1,len(trs[0].find_all('th', scope = 'col')))]
            index = [trs[i].find_all('td', scope = 'row')[0].get_text() for i in range(1,len(trs))]

            rates = [[float(trs[i].find_all('td')[j].get_text()) for j in range(1,len(col)+1)] for i in range(1,len(trs))]
            data = pd.DataFrame(rates, columns = col, index = index)
            data = data/100 #pct_value

            date = data.index.values[len(data)-1] #latest data

            self.rf_rate = data.loc[date][tenor]
        else:
            print('Failure')

        return self.rf_rate #risk free rate

    
    def BS_model(self):
        import scipy.stats
        import numpy as np
        from math import sqrt, exp
        import pandas as pd

        stock_price, var = self.get_stock_info() # from get_stock_info
        strike_price = self.strike_price
        r_f = self.get_rf()
        ttm =self.ttm
        
        norm_dist = scipy.stats.norm(0, 1)
        d1 = 1/(var * sqrt(ttm)) * (np.log(stock_price/strike_price) +(r_f + (var**2/2))*ttm)
        d2 = d1 - var*sqrt(ttm)

        call_value = norm_dist.cdf(d1) * stock_price - norm_dist.cdf(d2) * strike_price * exp(-r_f * ttm)
        put_value = strike_price * exp(-r_f * ttm) - stock_price + call_value

        delta_call = norm_dist.cdf(d1)
        delta_put = norm_dist.cdf(d1) - 1
        gamma = norm_dist.pdf(d1)/(stock_price * var * sqrt(ttm))
        vega = stock_price * norm_dist.pdf(d1) * sqrt(ttm)
        theta_call = -(stock_price * norm_dist.pdf(d1) * var)/(2*sqrt(ttm))- (r_f * strike_price * 
                                                                              norm_dist.cdf(d2) * exp(-r_f*ttm))
        theta_put = -(stock_price * norm_dist.pdf(d1) * var)/(2*sqrt(ttm))+ (r_f * strike_price * 
                                                                              norm_dist.cdf(-d2) * exp(-r_f*ttm))
        rho_call = strike_price * ttm * norm_dist.cdf(d2) * exp(-r_f * ttm)
        rho_put = -strike_price * ttm * norm_dist.cdf(-d2) * exp(-r_f * ttm)

        price = [call_value, put_value]
        delta = [delta_call, delta_put]
        gamma = [gamma]*2
        vega = [vega]*2
        theta = [theta_call, theta_put]
        rho = [rho_call, rho_put]

        output = [price, delta, gamma, vega, theta, rho]
        col = ['Call', 'Put']
        index = ['Price', 'Delta', 'Gamma', 'Vega', 'Theta', 'Rho']

        self.BS_df = pd.DataFrame(output, columns = col, index = index)
        
        return self.BS_df 

        
    
    def BS_sensitivity_plot(self):
#         stock_price = self.price
#         var = self.std ** 2
#         strike_price = self.strike_price
#         r_f = self.r_f
#         ttm =self.ttm
        option_type = self.option_type
        
        
        
    def mc_euro_options(self, option_type,S,K,T,r,v,simulations):
        
        option_type = self.option_type
        S, v = self.get_stock_info()
        K = self.strike_price
        r = self.get_rf()
        
        from random import gauss
        import math
        from math import exp, sqrt
    
        payoff_sum = 0
    
        for j in range(simulations):
            st = S * exp((r - 0.5 * v**2) * T + v * sqrt(T) * gauss(0,1.0))
            if option_type == 'c':
                payoff = max(0,st-K)
            elif option_type == 'p':
                payoff = max(0,K-st)
            payoff_sum += payoff
        
        price_T = (payoff_sum/float(simulations))*exp(-r * T)
        return price_T
    
        

    
    def simulations_converge(option_type,S,K,T,r,v,simulations):
        
        option_type = self.option_type
        S, v = self.get_stock_info()
        K = self.strike_price
        r = self.get_rf()
        
        
        import matplotlib.pyplot as plt
    
    
        simulation_path = []
        for simulation in range(simulations)[1:]:
            price_T_simulation = mc_euro_options(option_type,S,K,T,r,v,simulation)
            simulation_path.append(price_T_simulation)
        
        plt.plot(simulation_path)
    
        plt.xlabel('simulation times')
        plt.ylabel('option price')
    
        plt.show()
        
    
    def stock_time(self, S,K,r,v,simulations,steps,time_delta):
        
        S, v = self.get_stock_info()
        K = self.strike_price
        r = self.get_rf()
        
        
        from random import gauss
        import math
        from math import exp, sqrt
        import matplotlib.pyplot as plt
    
        steps = 252
        time_delta = T /steps
    
    
        for j in range(simulations):
            price_path = [S]
            St = S
            for t in range(steps):
                St = St * exp((r - 0.5 * v**2)* time_delta  + v * sqrt(time_delta) * gauss(0,1.0))
                price_path.append(St)
         
        plt.plot(price_path)
        
        plt.ylabel('stock price',fontsize=15)
        plt.xlabel('steps',fontsize=15)
        plt.show()
    
#     def BinomialTreeCRR(self, T, S, K, r, sigma, n, option_type='C'):
#         pass

In [67]:
O = OptionPricing('AAPL', 170, 1 )
O.BS_sensitivity_plot()

KeyboardInterrupt: 